<a href="https://colab.research.google.com/github/Joice-Almeida/projeto-final-desenvolve/blob/main/petroleo_gas_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingestão de Dados(integração com a tabela do bigquery)

In [ ]:
!pip install --upgrade google-cloud-bigquery  # instalando a biblioteca do bigquery


In [ ]:
# autenticação de usuário
from google.colab import auth
auth.authenticate_user()

In [ ]:
# importando a biblioteca do bigquery
from google.cloud import bigquery

# importando as bibliotecas do python
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

idProjeto = "t1engenhariadados" # id do projeto no bigquery

cliente = bigquery.Client(project = idProjeto) # criando um cliente que chama o projeto que contém a tabela do bigquery

In [ ]:
# Criando uma query que seleciona toda as linhas e colunas da tabela do bigquery, criando e lendo um dataframe
query = """

SELECT * FROM `t1engenhariadados.projeto_final_3_3.petroleo_gas`;


"""

resultado = cliente.query(query)
df = resultado.to_dataframe()
df.head()


In [ ]:
# Visualização inicial dos dados
df.info()
df.describe()

In [ ]:
# Verifica a quantidade de linhas e colunas e imprime
df.shape
linhas, colunas = df.shape[0] , df.shape[1]
print(f'Linhas - {linhas}')
print(f'Colunas - {colunas}')

In [ ]:
# Informa os nomes da colunas
df.columns

In [ ]:
# Mostra a contagem de valores nulos por coluna
df.isnull().sum()



In [ ]:
# Mostra os valores únicos em determinada dimensão
df[df.isnull().any(axis=1)]

In [ ]:
# Conta o número de linhas duplicadas
df.duplicated().sum()

# Mostra as linhas duplicadas (se houver)
df[df.duplicated()]

In [ ]:
# PREPARAÇÃO DOS DADOS
# 1.Converter col 'Production Date' para o formato de data (datetime),
# permitindo extrair info como ano, mês, etc.
df['Production Date'] = pd.to_datetime(df['Production Date'])
# 2. Criar uma coluna 'Ano' para facilitar o agrupamento por ano
df['Ano'] = df['Production Date'].dt.year
df['Mes'] = df['Production Date'].dt.month


df.head()

**Análise Classes de Terreno**

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Land Class'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Land Class'].value_counts(normalize=True) * 100)

In [ ]:
df['Land Class'].value_counts().plot(kind='bar',figsize =(6,4) )
plt.yticks(fontsize=10)
plt.xticks(fontsize=10, rotation=45, ha='right')
plt.title('Distribuição de Classes de Solo', fontsize=12)
plt.xlabel('Classe de Solo', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.tight_layout()
plt.show()


existem 3 tipos de classes de terrenos:
Federal, Mixed Exploratory e Native American

**Análise Classes de Terreno**

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Land Category'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Land Category'].value_counts(normalize=True) * 100)

Análise referente as regiões marítimas onde ocorre produção de recursos

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Offshore Region'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Offshore Region'].value_counts(normalize=True) * 100)

Estados e Condados

In [ ]:
df['State'].nunique()
df['State'].value_counts()

In [ ]:
df['County'].nunique()
df['County'].value_counts()

Recursos extraídos

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Commodity'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Commodity'].value_counts(normalize=True) * 100)

Produção de Recursos Anual

In [ ]:
analise_volume = df.groupby(['Ano', 'Commodity'])['Volume'].sum()
print(analise_volume)

In [ ]:
analise_volume.unstack().plot(kind='bar', figsize=(6,4), color=['#03045e', '#00b4d8'])
plt.xlabel('Ano',fontsize=10)
plt.yticks(fontsize=10 )
plt.xticks(fontsize=10)
plt.ylabel('Volume Total em Toneladas',fontsize=10)
plt.title('Produção de Recursos Anual',fontsize=10)

Quantidade de recursos produzida por mês

In [ ]:

# Lista com os nomes dos meses
nomes_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Agrupamento por mês
dados_grafico = df.groupby('Mes')['Volume'].sum().sort_index()

# Gráfico
plt.figure(figsize=(6, 4))
plt.plot(dados_grafico.index, dados_grafico.values, marker='o')
plt.xticks(ticks=range(1, 13), labels=nomes_meses, fontsize=12)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Volume Total em Toneladas', fontsize=12)
plt.title('Quantidade Produzida por Mês', fontsize=12)
plt.yticks(fontsize=12)
plt.grid(False)
plt.tight_layout()
plt.show()


Perfil de produção para cada Classe territorial


In [ ]:
analise = df.groupby(['Land Class', 'Commodity'])['Volume'].sum()
print(analise)

In [ ]:
analise.sort_values(ascending=False).unstack().plot(kind='barh', figsize=(6,4), color=['#0077b6', '#00b4d8'])
plt.yticks(fontsize=10 )
plt.xticks(fontsize=10)
plt.xlabel('Classe de Solo', fontsize=12)
plt.ylabel('Volume Total em Toneladas', fontsize=12)
plt.title('Perfil de Produção por Classe de Solo', fontsize=12)


Produção entre condados e estados

In [ ]:
on_off = df.groupby(['State', 'County'])['Volume'].sum()
print(on_off)

In [ ]:
df.groupby(['Commodity', 'State', 'Ano'])['Volume'].sum()

Ano e Estado com maior produção de Gas

In [ ]:
df_gas = df[df['Commodity'] == 'Gas (Mcf)']
gas_production = df_gas.groupby(['Ano', 'State'])['Volume'].sum().reset_index()
max_gas_production = gas_production.loc[gas_production['Volume'].idxmax()]
print(max_gas_production)


Ano e Estado com Maior produção de Oléo

In [ ]:
df_oil = df[df['Commodity'] == 'Oil (bbl)']
oil_production = df_oil.groupby(['Ano', 'State'])['Volume'].sum().reset_index()
max_oil_production = oil_production.loc[oil_production['Volume'].idxmax()]
print(max_oil_production)

Ano e Estado com menor produção de Gas

In [ ]:
df_gas = df[df['Commodity'] == 'Gas (Mcf)']
gas_production = df_gas.groupby(['Ano', 'State'])['Volume'].sum().reset_index()
min_gas_production = gas_production.loc[gas_production['Volume'].idxmin()]
print(min_gas_production)


Ano e Estado com menor produção de Oléo

In [ ]:
df_oil = df[df['Commodity'] == 'Oil (bbl)']
oil_production = df_oil.groupby(['Ano', 'State'])['Volume'].sum().reset_index()
min_oil_production = oil_production.loc[oil_production['Volume'].idxmin()]
print(min_oil_production)

Participação percentual de cada commodity na produção total de cada ano

In [ ]:
# Agrupar por Ano e Commodity e somar os volumes
# Isso cria um novo DataFrame com o volume total para cada commodity em cada ano.
comm_ano = df.groupby(['Ano', 'Commodity'])['Volume'].sum().reset_index()

# Passo 2: Calcular o volume total para cada ano (agora no DataFrame agregado)
comm_ano['total_volume_ano'] = comm_ano.groupby('Ano')['Volume'].transform('sum')

# Passo 3: Calcular a participação percentual correta
comm_ano['percentual'] = (comm_ano['Volume'] / comm_ano['total_volume_ano']) * 100

# Passo 4: Formatar e limpar para o resultado final
comm_ano['percentual'] = comm_ano['percentual'].map('{:.2f}%'.format)
comm_ano = comm_ano.drop(columns=['total_volume_ano'])

# Exibe o DataFrame final com o insight correto
print("Participação Percentual TOTAL de Cada Commodity por Ano:")
print(comm_ano)

### Média Anual de produção de oléo e gás

In [ ]:
df_oleo = df[df['Commodity'] == 'Oil (bbl)']
df_gas = df[df['Commodity'] ==  'Gas (Mcf)']

media_oleo = df_oleo.groupby('Ano')['Volume'].mean().round(2)
media_gas = df_gas.groupby('Ano')['Volume'].mean().round(2)

print("Média produção anual de oléo \n",media_oleo)
print("Média produção anual de gás \n",media_gas)

In [ ]:
# Salvando em csv
df.to_csv('petroleo_gas_tratado.csv', index=False)

In [ ]:
# Baixar em csv
from google.colab import files
files.download('petroleo_gas_tratado.csv')